In [6]:
import datetime
import pandas as pd
import gzip
import matplotlib.pyplot as plt
import seaborn as sns
import statistics


''' this function parses a file that contains the degrees of the nodes and returns nodes with degree 
satisfying two particular threshold: the minimum total degree and and the minimum percentage of random contacts'''
def get_stats(input_file, min_contact, min_rand_pct): 
    ''' f = open('net-daily/links-deg.txt', "r") '''
    f = open(input_file, "r")
    heavy_hitters = []
    for line in f:
        line = line.rstrip("\n")
        line_split = line.split()
        dev_id = int(line_split[0])
        rand  = int(line_split[2])
        close  = int(line_split[1])
        total  = int(line_split[3])
        pct_un = 100*rand/total
        if min_contact <= total and min_rand_pct<= pct_un:
            heavy_hitters.append(dev_id)
    return heavy_hitters

''' get the type of different social links i.e. frequent or random'''


def get_link_classes(all_links, classified_links): 

    ''' f = open('net-daily/links-deg.txt', "r") '''
    f = open(all_links, "r")
    frequent = {}
    random = {}
    for line in f:
        line = line.rstrip("\n")
        line_split = line.split('\t')
        dev_a = int(line_split[0])
        dev_b = int(line_split[1])
        count = int(line_split[2])
        days_a = int(line_split[7])
        days_b = int(line_split[8])
        overlap = int(line_split[9])
        if days_a<7 or days_b<7 or overlap<7:
            continue
        if count>1:
            if dev_a in frequent:
                frequent[dev_a][dev_b] = 1
            else:
                frequent[dev_a] = {}
                frequent[dev_a][dev_b] = 1
            if dev_b in frequent:
                frequent[dev_b][dev_a] = 1
            else:
                frequent[dev_b] = {}
                frequent[dev_b][dev_a] = 1
    f.close()
    f = open(classified_links, "r")
    for line in f:
        line = line.rstrip("\n")
        line_split = line.split('\t')
        dev_a = int(line_split[0])
        dev_b = int(line_split[1])
        ind = int(line_split[2])
        if ind == 1:
            if dev_a in frequent:
                frequent[dev_a][dev_b] = 1
            else:
                frequent[dev_a] = {}
                frequent[dev_a][dev_b] = 1
            if dev_b in frequent:
                frequent[dev_b][dev_a] = 1
            else:
                frequent[dev_b] = {}
                frequent[dev_b][dev_a] = 1
        else:
            if dev_a in random:
                random[dev_a][dev_b] = 1
            else:
                random[dev_a] = {}
                random[dev_a][dev_b] = 1
            if dev_b in random:
                random[dev_b][dev_a] = 1
            else:
                random[dev_b] = {}
                random[dev_b][dev_a] = 1
    f.close()
    return frequent,random 

'''parses the contact data to produce a daily contact time series per device
   that is the number of random and frequent close contacts per each device per day.
   It takes three paramters: a dictionary of frequent close contacts, a dictionary of random contacts and a set of files 
   that show when each contact took place 
'''
def daily_stats(known_contacts, unknown_contacts, day_files):
    days_mapping = {}
    known_per_day = {}
    uknown_per_day = {}
    f = open(day_files, "r")
    for file_det in f:
        file_det = file_det.rstrip("\n")
        file_det_split = file_det.split('\t')
        file_name = file_det_split[0]
        day_of_week = file_det_split[1]
        name_split = file_name.split('-')
        calendar_date = name_split[1]
        days_mapping[calendar_date] = day_of_week
        fd = open(file_name, "r")
        for line in fd:
            line = line.rstrip("\n")
            info = line.split('\t')
            dev_a = int(info[0])
            dev_b = int(info[1])
            if dev_a in known_contacts:
                if dev_b in known_contacts[dev_a]:
                    if dev_a in known_per_day:
                        if calendar_date in known_per_day[dev_a]:
                            known_per_day[dev_a][calendar_date] += 1
                        else:
                            known_per_day[dev_a][calendar_date] = 1
                    else:
                        known_per_day[dev_a] = {}
                        known_per_day[dev_a][calendar_date] = 1
            if dev_a in unknown_contacts:
                if dev_b in unknown_contacts[dev_a]:
                    if dev_a in uknown_per_day:
                        if calendar_date in uknown_per_day[dev_a]:
                            uknown_per_day[dev_a][calendar_date] += 1
                        else:
                            uknown_per_day[dev_a][calendar_date] = 1
                    else:
                        uknown_per_day[dev_a] = {}
                        uknown_per_day[dev_a][calendar_date] = 1
        fd.close()
    return days_mapping, known_per_day, uknown_per_day     

''' this function returns stats on random contacts per day'''
def find_day_stats(random, uknown_per_day,heavy_hitters):
    dev_rand = {}
    max_rand = {}
    max_date = {}
    median_rand = {}
    mean_rand = {}
    sf_rand = {}
    nin_rand = {}
    for dev in heavy_hitters:
        if dev in random:
            total_random = len(random[dev])
            if dev in uknown_per_day:
                sample = [] 
                max_day = ""
                max_count = 0
                for day in uknown_per_day[dev]:
                    sample.append(uknown_per_day[dev][day])
                    if max_count < uknown_per_day[dev][day]:
                        max_count = uknown_per_day[dev][day]
                        max_day = day
                if max_count>0:
                    dev_rand[dev] = total_random
                    max_rand[dev] = max_count
                    max_date[dev] = max_day
                if len(sample) < 10:
                    continue  
                median_val = statistics.median(sample)
                mean_val = statistics.mean(sample)
                nn = statistics.quantiles(sample, n=10)
                sf = statistics.quantiles(sample, n=4)
                median_rand[dev] = median_val
                mean_rand[dev] = mean_val  
                sf_rand[dev] = sf[2]
                nin_rand[dev] = nn[8]
    return dev_rand, max_rand, max_date, median_rand, mean_rand, sf_rand, nin_rand  


''' this function returns stats on contacts per day for highly connected nodes. It takes thrree input paramteres:
dictionary of knnown close contacts, dictonary of random contacts and list of highly connecetd nodes.'''
def find_day_stats_all(known_per_day, uknown_per_day,heavy_hitters):
    max_all = {}
    median_all = {}
    mean_all = {}
    sf_all = {}
    nin_all = {}
    for dev in heavy_hitters:
        sample = []
        visited = 0
        if dev in uknown_per_day:
            visited = 1
            for day in uknown_per_day[dev]:
                sample.append(uknown_per_day[dev][day])
                if dev in known_per_day:
                    if day in known_per_day[dev]:
                        sample[len(sample)-1] = sample[len(sample)-1] + known_per_day[dev][day]
        if dev in known_per_day:
            if visited:
                for day in known_per_day[dev]:
                    if day not in uknown_per_day[dev]:
                        sample.append(known_per_day[dev][day])
            else:
                for day in known_per_day[dev]:
                    sample.append(known_per_day[dev][day])
        ''' we need at least 10 data points to calculate the 90th percentile'''
        if len(sample) < 10:
            continue  
        median_val = statistics.median(sample)
        mean_val = statistics.mean(sample)
        max_val = max(sample)
        nn = statistics.quantiles(sample, n=10)
        sf = statistics.quantiles(sample, n=4)
        max_all[dev] = max_val
        median_all[dev] = median_val
        mean_all[dev] = mean_val  
        sf_all[dev] = sf[2]
        nin_all[dev] = nn[8]
    return max_all, median_all, mean_all, sf_all, nin_all  





''' this function shows the distribution of contact type for heavy hitters in the whole time period 
and by day of week. A skew towards weekend underscore social activites. Also specific dates can reveal details about
contact type'''
def find_day_dist(days_mapping, known_per_day, unknown_per_day,heavy_hitters):
    unknown_ts = {}
    known_ts = {}
    unknown_dow = {}
    known_dow = {}
    total_known = 0
    total_unknown = 0
    for dev in heavy_hitters:
        if dev in known_per_day:
            for date in known_per_day[dev]:
                dow = days_mapping[date]
                total_known += known_per_day[dev][date]
                if date in known_ts:
                    known_ts[date] += known_per_day[dev][date]
                else:
                    known_ts[date] = known_per_day[dev][date]
                if dow in known_dow:
                    known_dow[dow] += known_per_day[dev][date]
                else:
                    known_dow[dow] = known_per_day[dev][date]
        if dev in unknown_per_day:
            for date in uknown_per_day[dev]:
                dow = days_mapping[date]
                total_unknown += unknown_per_day[dev][date]
                if dow in unknown_dow:
                    unknown_dow[dow] += unknown_per_day[dev][date]
                else:
                    unknown_dow[dow] = unknown_per_day[dev][date]
                if date in unknown_ts:
                    unknown_ts[date] += unknown_per_day[dev][date]
                else:
                    unknown_ts[date] = unknown_per_day[dev][date]       
    return total_known, total_unknown, unknown_ts, known_ts, unknown_dow, known_dow     

''' this function measures the liklehood a heavy hitter is connected to another heavy hitter
a high probability indicate communities of heavy hitters, while a low probability may indicate potential
super spreaders '''
def find_hh_similarity(heavy_hitters, known_contacts, unknown_contacts):
    known_similarity = {}
    unknown_similarity = {}
    all_similarity = {}
    all_stats = {}
    known_stats = {}
    unknown_stats = {}
    for dev in heavy_hitters:
        if dev in known_contacts:
            all_stats[dev] = len(known_contacts[dev])
            known_stats[dev] = len(known_contacts[dev])
            for dev_x in known_contacts[dev]:
                if dev_x in heavy_hitters:
                    if dev in known_similarity:
                        known_similarity[dev] += 1
                        all_similarity[dev] += 1
                    else:
                        known_similarity[dev] = 1
                        all_similarity[dev] = 1
        if dev in unknown_contacts:
            if dev in all_stats:
                all_stats[dev] += len(unknown_contacts[dev])
            else:
                all_stats[dev] = len(unknown_contacts[dev])
            unknown_stats[dev] = len(unknown_contacts[dev])
            for dev_x in unknown_contacts[dev]:
                if dev_x in heavy_hitters:
                    if dev in all_similarity:
                        all_similarity[dev] += 1
                    else:
                        all_similarity[dev] = 1
                    if dev in unknown_similarity:
                        unknown_similarity[dev] += 1
                    else:
                        unknown_similarity[dev] = 1
    return all_stats, known_stats, unknown_stats, all_similarity, known_similarity, unknown_similarity  

''' this function producces figure 3e in the main text '''
def highly_connected_users_neighbourhood(all_links, classified_links,degree_per_node):
    random, frequent = get_link_classes(all_links, classified_links)
    hh_50_0 = get_stats(degree_per_node,50,0)
    all_stats, known_stats, unknown_stats, all_similarity, known_similarity, unknown_similarity = find_hh_similarity(hh_50_0, frequent, random)
    sim_all = []
    for dev in all_stats:
        if dev in all_similarity:
            sim_all.append(all_similarity[dev])
        else:
            sim_all.append(0)
    f, p1 = plt.subplots(figsize=(7, 3.5))
    p1.set_xlabel("Number of highly connected neighbours",fontsize=18)
    p1.set_ylabel("Frequency", fontsize=18)
    plt.hist(sim_all, density=False, bins=50, rwidth=0.85)  

''' this function produces figure 3f in the main text '''
def get_ninetieth_pct_day_stats(all_links, classified_links,day_files,degree_per_node):
    random, frequent = get_link_classes(all_links, classified_links)
    days_mapping, known_per_day, uknown_per_day = daily_stats(frequent, random, day_files)
    hh_50_0 = get_stats(degree_per_node,50,0)
    max_all, median_all, mean_all, sf_all, nin_all = find_day_stats_all(known_per_day, uknown_per_day,hh_50_0)
    max_per_dev = []
    nth_pct_per_dev = [] 
    first_range = []
    second_range = [] 
    third_range = []
    fourth_range = []
    for dev in nin_all:
        max_per_dev.append(max_all[dev])
        nth_pct_per_dev.append(nin_all[dev])
    for i in range(len(max_per_dev)):
        if max_per_dev[i]<=10 :
            first_range.append(nth_pct_per_dev[i])
        else:
            if max_per_dev[i]<=20:
                second_range.append(nth_pct_per_dev[i])
            else:
                if max_per_dev[i]<=40:
                    third_range.append(nth_pct_per_dev[i])
                else:
                    fourth_range.append(nth_pct_per_dev[i])
    fig, ax = plt.subplots(figsize=(8, 4))
    plt.xticks(rotation = 45)
    ax.yaxis.grid()
    ax.set_ylabel("The 90th percentile day",fontname='DejaVu Sans',fontsize=18)
    ax.set_xlabel("Range of max daily contacts (M)",fontname='DejaVu Sans',fontsize=18)
    data = [first_range, second_range, third_range, fourth_range]
    labels = [r'$M \leq 10$', r'$10 < M \leq 20$', r'$20 < M \leq 40$', r'$40 < M$']
    box = ax.boxplot(data, notch=True, patch_artist=True, vert=True, labels=labels)
    colors = ['brown', 'brown', 'brown', 'brown']
    for patch, color in zip(box['boxes'], colors):
        patch.set_facecolor(color)
        
        
''' this function produces supplementery figure 8 '''
    
def similarity_between_highly_connected_users(all_links, classified_links,degree_per_node):
    random, frequent = get_link_classes(all_links, classified_links)
    hh_50_0 = get_stats(degree_per_node,50,0)
    all_stats, known_stats, unknown_stats, all_similarity, known_similarity, unknown_similarity = find_hh_similarity(hh_50_0, frequent, random)
    sim_ind = []
    for dev in all_stats:
        if dev in all_similarity:
            sim_ind.append(all_similarity[dev]/all_stats[dev])
        else:
            sim_ind.append(0)
    known_sim_ind = []
    for dev in known_similarity:
        if dev in known_similarity:
            known_sim_ind.append(known_similarity[dev]/known_stats[dev])
        else:
            known_sim_ind.append(0)
    unknown_sim_ind = []
    for dev in unknown_similarity:
        if dev in unknown_similarity:
            unknown_sim_ind.append(unknown_similarity[dev]/unknown_stats[dev])
        else:
            unknown_sim_ind.append(0)
    f, p1 = plt.subplots(figsize=(7, 3.5))
    p1.set_xlabel("Fraction of highly connected neighbours ",fontsize=18)
    p1.set_ylabel("Density (Unkown)", color="b",fontsize=18)
    p1.tick_params(labelsize=14)
    p1=sns.kdeplot(unknown_sim_ind, shade=True, color="b")
    ax2 = p1.twinx()  # instantiate a second axes that shares the same x-axis
    ax2.tick_params(labelsize=14)
    color = 'tab:red'
    ax2.set_ylabel('Density (Known)', color=color, fontsize=18)  # we already handled the x-label with ax1
    ax2=sns.kdeplot(known_sim_ind, shade=True, color="r")
    ax2.tick_params(axis='y', labelcolor=color)